In [1]:
!pip install ipywidgets --quiet
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import ipywidgets as widgets
from IPython.display import display, clear_output

df = pd.read_csv("/content/heineken_beer_cleaned.csv", parse_dates=["date"])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.5 MB/s eta 0:00:00


In [2]:
brewery_dropdown = widgets.Dropdown(
    options=sorted(df["brewery"].unique()),
    description="Brewery:"
)

sku_dropdown = widgets.Dropdown(
    options=sorted(df["product_name"].unique()),
    description="SKU:"
)

display(brewery_dropdown, sku_dropdown)


Dropdown(description='Brewery:', options=('ALLAGASH BREWING COMPANY', 'AMERICAN BEVERAGE CORPORATION', 'ANHEUS…

Dropdown(description='SKU:', options=('10 BARREL JOE IPA 4/6PK NR', '21ST AMENDMENT IPA 4/6 CAN - 12OZ', '21ST…

In [3]:
def update_dashboard(brewery, sku):
    clear_output(wait=True)
    display(brewery_dropdown, sku_dropdown)

    filtered = df[(df["brewery"] == brewery) & (df["product_name"] == sku)]

    if filtered.empty:
        print("No data for this selection.")
        return

    # KPIs
    total_sales = int(filtered["retail_volume"].sum())
    avg_monthly = int(filtered.groupby("date")["retail_volume"].sum().mean())
    total_wh = int(filtered["warehouse_volume"].sum())
    total_transfer = int(filtered["inventory_transfer"].sum())

    print("========== KPI PANEL ==========")
    print("Total Retail Sales:", total_sales)
    print("Avg Monthly Demand:", avg_monthly)
    print("Warehouse Volume:", total_wh)
    print("Transfers:", total_transfer)

    # Trend chart (Interactive)
    trend = filtered.groupby("date")["retail_volume"].sum().reset_index()
    fig = px.line(trend, x="date", y="retail_volume",
                  title="Monthly Demand Trend", markers=True)
    fig.show()

    # Time Series
    ts = trend.set_index("date")["retail_volume"].asfreq("MS").interpolate()

    # SARIMA
    model = SARIMAX(ts,
                    order=(1,1,1),
                    seasonal_order=(1,1,1,12),
                    enforce_stationarity=False,
                    enforce_invertibility=False)
    results = model.fit(disp=False)
    forecast = results.forecast(steps=6)

    # Forecast plot
    plt.figure(figsize=(10,4))
    plt.plot(ts, label="History")
    plt.plot(forecast, linestyle="--", marker="o", label="Forecast")
    plt.legend()
    plt.grid(True)
    plt.title("6-Month Forecast")
    plt.show()

    # Forecast table
    display(pd.DataFrame({
        "Date": forecast.index,
        "Forecast": forecast.values.round(0)
    }))

    # Inventory risk
    filtered["gap"] = filtered["warehouse_volume"] - filtered["retail_volume"]
    risk = filtered.sort_values("gap")

    print("------ OVERSTOCK RISK ------")
    display(risk.tail(5))

    print("------ UNDERSTOCK RISK ------")
    display(risk.head(5))


widgets.interact(update_dashboard, brewery=brewery_dropdown, sku=sku_dropdown)


interactive(children=(Dropdown(description='Brewery:', options=('ALLAGASH BREWING COMPANY', 'AMERICAN BEVERAGE…

<function __main__.update_dashboard(brewery, sku)>

In [4]:
def update_sku_options(*args):
    sku_dropdown.options = sorted(df[df["brewery"] == brewery_dropdown.value]["product_name"].unique())

brewery_dropdown.observe(update_sku_options, names='value')
update_sku_options()
